In [1]:
import yfinance as yf
import pandas as pd
from ipywidgets import Output
from sklearn.metrics import precision_score, accuracy_score,recall_score

predictors = []

In [2]:
stocks = ["nvda",'qqq']
data =pd.DataFrame([])
# pd.options.display.float_format = '{:.2f}'.format
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
first = True
predictors =[]
               
for stock in stocks:
    
    stk = yf.Ticker(stock)
    stock_data = stk.history(period='max') 
    stock_data=stock_data.loc["1990-1-1":].copy()
    
    Gap_column = f'{stock}_Gap'
    stock_data[Gap_column]=(stock_data['Open']-stock_data['Close'].shift(1))/stock_data['Close']
    predictors +=[Gap_column]
    
    if stock =="qqq":
        del (stock_data['Capital Gains'])
       
       
      
    del stock_data["Dividends"]
    del stock_data["Stock Splits"] 
    del stock_data["Open"]
    del stock_data["High"]
    del stock_data["Low"]
    stock_data["Tomarrow"] = stock_data["Close"].shift(-5)
    stock_data["Target"] = (stock_data["Close"]<stock_data["Tomarrow"]).astype(int)
    
    
 
    
    horizons = [2,5,60,250,1000]
    # predictors = []

    for horizon in horizons:
        rolling_averages = stock_data.rolling(horizon).mean()

        ratio_column = f"{stock}_Close_Avg_{horizon}"
        stock_data[ratio_column] = stock_data["Close"] / rolling_averages["Close"]

        trend_column = f"{stock}_Trend_{horizon}"
        stock_data[trend_column] = stock_data.shift(1).rolling(horizon).sum()["Target"]
         
        
        v_ratio_col =f"{stock}_VolAvg_{horizon}"
        stock_data[v_ratio_col] = stock_data["Volume"]/rolling_averages["Volume"]
        
        predictors+= [ratio_column, trend_column,v_ratio_col]
        
    
    if  first:
        first = False
        target_stock = stock
    else:
        del stock_data['Close']
        del stock_data['Volume']
        del stock_data['Target']
        del stock_data['Tomarrow']
    data = pd.concat([data,stock_data],ignore_index = False,axis=1)
    

In [3]:
data=data.dropna()

,Close,Volume,nvda_Gap,Tomarrow,Target,nvda_Close_Avg_2,nvda_Trend_2,nvda_VolAvg_2,nvda_Close_Avg_5,nvda_Trend_5,nvda_VolAvg_5,nvda_Close_Avg_60,nvda_Trend_60,nvda_VolAvg_60,nvda_Close_Avg_250,nvda_Trend_250,nvda_VolAvg_250,nvda_Close_Avg_1000,nvda_Trend_1000,nvda_VolAvg_1000,qqq_Gap,qqq_Close_Avg_2,qqq_Trend_2,qqq_VolAvg_2,qqq_Close_Avg_5,qqq_Trend_5,qqq_VolAvg_5,qqq_Close_Avg_60,qqq_Trend_60,qqq_VolAvg_60,qqq_Close_Avg_250,qqq_Trend_250,qqq_VolAvg_250,qqq_Close_Avg_1000,qqq_Trend_1000,qqq_VolAvg_1000
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003-03-04 00:00:00-05:00,0.940999,53839200,0.003249,0.934884,0,0.995954,0.0,0.868075,0.989231,1.0,0.623721,1.030485,19.0,0.508719,0.609990,91.0,0.441005,0.484246,491.0,0.667981,0.000408,0.996948,0.0,0.962468,0.993512,1.0,0.956135,0.972087,21.0,0.838080,0.911792,95.0,0.643215,0.450067,479.0,1.047689
2003-03-05 00:00:00-05:00,0.956288,52215600,-0.009592,0.960874,1,1.008058,0.0,0.984691,1.002083,1.0,0.638140,1.049570,18.0,0.501856,0.625366,91.0,0.428864,0.491972,491.0,0.647461,-0.004073,1.001019,0.0,1.104307,0.992882,1.0,1.143360,0.975135,21.0,1.037371,0.915442,95.0,0.793622,0.451208,478.0,1.290272
2003-03-06 00:00:00-05:00,0.940235,54806400,-0.008130,1.056427,1,0.991536,1.0,1.024208,0.989541,1.0,0.691305,1.034599,19.0,0.531965,0.620575,92.0,0.451214,0.483577,491.0,0.679201,-0.004900,0.998776,0.0,0.922094,0.992704,0.0,1.002354,0.974028,21.0,0.892470,0.915140,95.0,0.679695,0.450317,477.0,1.102726
2003-03-07 00:00:00-05:00,0.954759,58114800,-0.015212,1.025849,1,1.007664,2.0,1.029298,1.006933,2.0,1.004822,1.051332,20.0,0.567646,0.635952,93.0,0.478813,0.490908,491.0,0.719780,-0.016707,1.001020,1.0,1.107118,0.999756,1.0,1.152541,0.976458,22.0,1.107322,0.918931,96.0,0.842693,0.451460,477.0,1.365607
2003-03-10 00:00:00-05:00,0.928005,34479600,-0.010708,1.085473,1,0.985790,2.0,0.744745,0.982996,3.0,0.680190,1.024675,20.0,0.342614,0.623522,94.0,0.284935,0.477021,492.0,0.426906,-0.008747,0.989083,2.0,0.865807,0.983291,2.0,0.892348,0.956377,22.0,0.848205,0.900972,97.0,0.644075,0.441936,478.0,1.041470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-27 00:00:00-05:00,786.973083,39170500,0.003672,859.640015,1,0.997522,2.0,0.874656,1.028449,5.0,0.597027,1.355416,49.0,0.844953,1.812977,162.0,0.827071,3.364955,592.0,0.841150,0.002559,1.001201,2.0,1.010036,1.006157,5.0,0.800113,1.057021,49.0,0.753709,1.198562,167.0,0.658322,1.370281,596.0,0.638970
2024-02-28 00:00:00-05:00,776.593567,39311000,-0.013919,887.000000,1,0.993362,2.0,1.001790,0.988554,5.0,0.658856,1.325782,50.0,0.852062,1.780048,162.0,0.830507,3.310447,593.0,0.844568,-0.004985,0.997331,1.0,0.988381,0.996374,4.0,0.851409,1.049389,49.0,0.741896,1.190276,166.0,0.644477,1.361934,596.0,0.625283
2024-02-29 00:00:00-05:00,791.082886,50728900,0.018088,926.690002,1,1.009243,2.0,1.126809,1.005529,5.0,0.966094,1.338200,51.0,1.094071,1.804029,162.0,1.070662,3.361807,594.0,1.090045,0.006743,1.004266,1.0,1.126686,1.004485,4.0,1.167166,1.056258,49.0,0.956402,1.198543,166.0,0.832177,1.372574,597.0,0.807584


In [4]:
import sklearn
from sklearn.ensemble import RandomForestClassifier

train = data.iloc[:-100]
test = data.iloc[-100:]

In [5]:
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []
    
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [6]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >=.6] = 1
    preds[preds <.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [7]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

In [8]:
predictions = backtest(data, model, predictors)

In [9]:
recall=recall_score(predictions["Target"], predictions["Predictions"])
prec = precision_score(predictions["Target"], predictions["Predictions"])
acc = accuracy_score(predictions["Target"], predictions["Predictions"])

In [10]:
feature_imp=pd.Series( model.feature_importances_).sort_index(ascending=True)

In [11]:
pred=pd.Series(predictors)

In [12]:
predictors2=data.tail(1)
predictors2

,Close,Volume,nvda_Gap,Tomarrow,Target,nvda_Close_Avg_2,nvda_Trend_2,nvda_VolAvg_2,nvda_Close_Avg_5,nvda_Trend_5,nvda_VolAvg_5,nvda_Close_Avg_60,nvda_Trend_60,nvda_VolAvg_60,nvda_Close_Avg_250,nvda_Trend_250,nvda_VolAvg_250,nvda_Close_Avg_1000,nvda_Trend_1000,nvda_VolAvg_1000,qqq_Gap,qqq_Close_Avg_2,qqq_Trend_2,qqq_VolAvg_2,qqq_Close_Avg_5,qqq_Trend_5,qqq_VolAvg_5,qqq_Close_Avg_60,qqq_Trend_60,qqq_VolAvg_60,qqq_Close_Avg_250,qqq_Trend_250,qqq_VolAvg_250,qqq_Close_Avg_1000,qqq_Trend_1000,qqq_VolAvg_1000
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-03-04 00:00:00-05:00,852.330017,61561600,0.021716,857.73999,1,1.017658,2.0,1.127096,1.057552,5.0,1.290872,1.411779,51.0,1.314324,1.922639,163.0,1.296637,3.598234,596.0,1.323451,0.0,0.998213,1.0,0.878627,1.008449,3.0,0.916417,1.063306,48.0,0.777198,1.208368,167.0,0.674261,1.38599,598.0,0.65418


In [13]:
del predictors2["Close"]
del predictors2["Volume"]
del predictors2["Tomarrow"]
del predictors2["Target"]


In [14]:
# res = model.predict(predictors2)

In [15]:
 print(pd.concat([pred,feature_imp],axis=1))

                      0         1
0              nvda_Gap  0.014724
1      nvda_Close_Avg_2  0.035770
2          nvda_Trend_2  0.331338
3         nvda_VolAvg_2  0.011692
4      nvda_Close_Avg_5  0.052740
5          nvda_Trend_5  0.093699
6         nvda_VolAvg_5  0.013227
7     nvda_Close_Avg_60  0.016799
8         nvda_Trend_60  0.014539
9        nvda_VolAvg_60  0.011015
10   nvda_Close_Avg_250  0.014360
11       nvda_Trend_250  0.009673
12      nvda_VolAvg_250  0.010822
13  nvda_Close_Avg_1000  0.017050
14      nvda_Trend_1000  0.011709
15     nvda_VolAvg_1000  0.011266
16              qqq_Gap  0.013314
17      qqq_Close_Avg_2  0.024096
18          qqq_Trend_2  0.092255
19         qqq_VolAvg_2  0.012861
20      qqq_Close_Avg_5  0.033294
21          qqq_Trend_5  0.024433
22         qqq_VolAvg_5  0.014149
23     qqq_Close_Avg_60  0.014957
24         qqq_Trend_60  0.010382
25        qqq_VolAvg_60  0.014574
26    qqq_Close_Avg_250  0.015233
27        qqq_Trend_250  0.009432
28       qqq_V

In [16]:
predictions["Predictions"].value_counts()

Predictions
0.0    1529
1.0    1258
Name: count, dtype: int64

In [17]:
print("baseline: ", predictions["Target"].value_counts() / predictions.shape[0])

baseline:  Target
1    0.59024
0    0.40976
Name: count, dtype: float64


In [18]:
print('accuracy: ',acc,' (TP + TN)/(TP + TN + FP +FN)' )
print('recall: ',recall,' TP/(TP+FN)')
print('precision: ',prec,'TP/(TP + FP)')

accuracy:  0.736275565123789  (TP + TN)/(TP + TN + FP +FN)
recall:  0.6589665653495441  TP/(TP+FN)
precision:  0.8616852146263911 TP/(TP + FP)


In [19]:

pred
res = model.predict(predictors2)


In [20]:
if res:
    print('trade')
else:
    print('stand down')
        

trade


In [21]:
def current_price(instrument):
    data = yf.Ticker(instrument).history(period="10m", interval="1m")
    return data["Close"].iloc[-1]
    

print(target_stock,": ", current_price(target_stock))

nvda :  857.9199829101562
